In [ ]:
# Import all libraries
import rasterio
from rasterio.plot import show
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from glob import glob
import os
import sys

### k-Nearest Neighbors Classification
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html


#### This code has been tested with Landsat 4 TM, Landsat 8 OLI, Landsat 7 ETM+, Sentinel 2 MSI, and Sentinel 3 OLCI optical data

In [ ]:
### open and read input raster file
ras = 'D:/your_input_directory/your_input_raster.tif'

In [ ]:
### read opened raster as numpy array
with rasterio.open(ras, 'r')  as src:
    show(src, cmap='viridis') # to visualize the raster
    ras_arr = src.read(1)      # to read raster as numpy array
### these steps are not mendatory
### but can run to see the info of input raster
    print(ras_arr.shape)       # to see the number of rows and columns
    print(ras_arr)             # to print the array
    print(np.min(ras_arr))     # to print minimum value of the array
    print(np.max(ras_arr))     # to print maximum value of the array

In [ ]:
### plot array without geocoordinate for 
### selecting supervised classes' (Y1:Y2, X1:X2)
## based on their array coordinate
plt.imshow(ras_arr, cmap="nipy_spectral")
### plot colorbar
plt.colorbar(shrink=0.5)
plt.clim(np.min(ras_arr), np.max(ras_arr))
plt.gcf().set_size_inches(9, 7)

In [ ]:
### make the supervised classes
### for test case five classes has been defined
### you can change the name of each class in 'class*'
### according to your needs
### class dictionary with keys and values
classes = {'class1': 0, 'class2': 1, 'class3': 2, 'class4': 3, 'class5': 4} 
n_classes = len(classes) # assign the lenght of class dictionary

In [ ]:
### build training data with data from define classes
### this train data will supervise input raster based on kNN algorithm
ras_knn = n_classes*np.ones(shape=(ras_arr.shape), dtype=np.int)
### fixed the area of each class on the raster
### technically on array
### y1,y2, and x1,x2 must be different for each class
### precision of y1,y2, and x1,x2 control the outcome of kNN
ras_knn[y1:y2, x1:x2] = classes['class1']
ras_knn[y1:y2, x1:x2] = classes['class2']
ras_knn[y1:y2, x1:x2] = classes['class3']
ras_knn[y1:y2, x1:x2] = classes['class4']
ras_knn[y1:y2, x1:x2] = classes['class5']

In [ ]:
### fix the shape and meta info for training and test set
X = ras_arr.reshape([-1, 1]) 
y = ras_knn.ravel()
train = np.flatnonzero(ras_knn < n_classes)
test = np.flatnonzero(ras_knn == n_classes)

### call and run kNN algorithm
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=5) ### number of neighbors can be changed based on needs
neigh.fit(X[train], y[train])
y[test] = neigh.predict(X[test])
ras_knn = y.reshape(ras_arr.shape) ### must reshape the output array for saving as geotiff later

In [ ]:
### plot kNN array for quick look
plt.imshow(ras_knn, cmap="Paired") # you can use any cmap from matplotlib cmap list
### plot colorbar
plt.colorbar(shrink=0.5)
plt.clim(np.min(ras_knn), np.max(ras_knn))
print(ras_knn.shape)
print(ras_knn)
print(np.min(ras_knn))
print(np.max(ras_knn))

### save array as jpg file
### make the output file directory
fig_direc = 'D:/your_output_directory/CI'
if not os.path.exists(fig_direc):
    os.makedirs(fig_direc)
### Save the figure as jpg format
plt.savefig('D:/your_output_directory/your_output_raster.jpg', dpi=600) # dpi can be changed according to 
# needs but 600 is high enough and such good resolution

In [ ]:
### save the output as GeoTiff file
### Get metadata from input raster
with rasterio.open(ras) as src:
    meta = src.meta
meta.update(dtype=rasterio.float32)
### Create output folder and write output file in it as geotiff
file = "D:/your_output_directory/your_output_raster.tif"
os.makedirs(os.path.dirname(file), exist_ok=True)
with rasterio.open(file, 'w', **meta) as dst:
    dst.write(ras_knn.astype(rasterio.float32), 1)